# Agilepy quickstart notebook

This notebook will show you how to use the Agilepy library.

## The output directory must deleted first

The path to the output directory is defined in the configuration file (more on that later).

In [1]:
import os
from pathlib import Path
from shutil import rmtree

In [2]:
outDir = Path(os.environ["AGILE"]).joinpath( Path("agilepy-test-data/unittesting-output/jupyter/") )

if outDir.exists() and outDir.is_dir():
    print("Removing",outDir)
    rmtree(outDir)

In [3]:
def readFile(filepath):
    with open(filepath, "r") as f:
        return f.readlines()

## Using the API class

We need to import the class in our scope

In [4]:
from agilepy.api.AGAnalysis import AGAnalysis

The constructor need two files:
* a configuration file (yaml)
* a file containing a list of sources (xml format, or AGILE txt format)

For this tutorial, we already wrote them. If you want to look at all the possible configuration options, check the [Agilepy's documentation](https://agilepy.readthedocs.io/en/latest/manual/configuration_file.html).

Run the next cell to read the minimal configuration file we wrote:

In [5]:
with open("./config_file.yaml") as f:
    print(f.read())

input:
  evtfile: $AGILE/agilepy-test-data/evt_index/agile_proc3_fm3.119_asdc2_EVT.index
  logfile: $AGILE/agilepy-test-data/log_index/agile_proc3_data_asdc2_LOG.log.index

output:
  outdir: $AGILE/agilepy-test-data/unittesting-output/jupyter
  filenameprefix: quickstart
  logfilenameprefix: quickstart
  verboselvl: 1

selection:
  tmin: 456361778
  tmax: 456537945
  timetype: TT
  glon: 80
  glat: 0



Run the next cell to read the xml file containing the descriptions of two sources:

In [6]:
with open("./sources.xml") as f:
    print(f.read())

<?xml version="1.0" ?>
<source_library title="source library">

  <source name="2AGLJ2021+4029" type="PointSource">
    <spectrum type="PLExpCutoff">
      <parameter name="flux" free="1"  value="119.3e-08"/>
      <parameter name="index" free="0" scale="-1.0" value="1.75" min="0.5" max="5"/>
      <parameter name="cutoffEnergy" free="0" value="3307.63" min="20" max="10000"/>
    </spectrum>
    <spatialModel type="PointSource" locationLimit="0">
      <parameter name="pos" value="(78.2375, 2.12298)" free="0" />
    </spatialModel>
  </source>

  <source name="2AGLJ2021+3654" type="PointSource">
    <spectrum type="PLExpCutoff">
      <parameter name="flux" free="1"  value="70.89e-08"/>
      <parameter name="index" free="0" scale="-1.0" value="1.38" min="0.5" max="5"/>
      <parameter name="cutoffEnergy" free="0" value="960.59" min="20" max="10000"/>
    </spectrum>
    <spatialModel type="PointSource" locationLimit="0">
      <parameter name="pos" value="(75.2562, 0.151831)" free="0

## Calling the constructor

You need to instantiate the class if you want to use it. 

The [constructor](https://agilepy.readthedocs.io/en/latest/api/analysis_api.html#api.AGAnalysis.AGAnalysis) will setup a console logger which verbosity is defined by the *'verboselvl'* configuration option. A file logger is activated as well and its verbosity is always set to the maximum value.

The user-defined configuration options will override the default ones. A validation step is then performed: in case of wrong/malformed configuration options an Exception will be raised.  

All the sources present in *sources.xml* are loaded in-memory inside an object of class *SourcesLibrary* which is a member of the *AGAnalysis* class. 

In [7]:
ag = AGAnalysis("./config_file.yaml", "./sources.xml")

2020-02-26 17:52:43,694 [INFO    ] [AgilepyLogger] File and Console loggers are active. Log file: /home/rt/.conda/envs/agilepy/agiletools/agilepy-test-data/unittesting-output/jupyter/logs/quickstart_20200226-175243.log
2020-02-26 17:52:43,697 [INFO    ] [PlottingUtils] Plot configuration: 'standard'. fig_width_pt: 426.791340 fontsize:9.000000


## Managing the sources 

### Loading the sources into the SourcesLibrary

You can load other sources after the AGAnalysis object initialization, with the [loadSource(sourcesFilepath)](https://agilepy.readthedocs.io/en/latest/api/analysis_api.html#api.AGAnalysis.AGAnalysis.loadSource) method. 

This method let you to specify a rangeDist argument to filter out the sources which distance from (glon, glat) is not in the rangeDist interval. 

In [8]:
ag.loadSources("./sources.xml", rangeDist = (0, 20))

2020-02-26 17:52:49,216 [WARNING ] [SourcesLibrary] The source 2AGLJ2021+4029 already exists. The 'sourceObject' will not be added to the SourcesLibrary.
2020-02-26 17:52:49,218 [WARNING ] [SourcesLibrary] The source 2AGLJ2021+3654 already exists. The 'sourceObject' will not be added to the SourcesLibrary.


[]

### Getting the available catalogs
You can also get the available AGILE catalogs with the [getSupportedCatalogs()](https://agilepy.readthedocs.io/en/latest/api/analysis_api.html#api.AGAnalysis.AGAnalysis.getSupportedCatalogs) method.

In [9]:
catalogs = ag.getSupportedCatalogs()
catalogs

['/home/rt/.conda/envs/agilepy/agiletools/catalogs/2AGL.multi']

### Converting a catalog from .txt format to .xml format 
If you want to work with the more convenient xml format, you can convert the source file from the txt format to the xml format, using the [convertCatalogToXml()](https://agilepy.readthedocs.io/en/latest/api/analysis_api.html#api.AGAnalysis.AGAnalysis.convertCatalogToXml) method.

In [10]:
ag.convertCatalogToXml(catalogs[0])

2020-02-26 17:52:54,950 [INFO    ] [SourcesLibrary] File /home/rt/.conda/envs/agilepy/agiletools/catalogs/2AGL.xml has been produced


'/home/rt/.conda/envs/agilepy/agiletools/catalogs/2AGL.xml'

## Generating maps

You can use the [generateMaps()](https://agilepy.readthedocs.io/en/latest/api/analysis_api.html#api.AGAnalysis.AGAnalysis.generateMaps) method to produce one or more sky maps of 4 different types (counts, exposure, gas and int) using the AGILE data. The method's behaviour is controlled by several configuration options. Check the [*'map'* section of the documentation](https://agilepy.readthedocs.io/en/latest/manual/configuration_file.html#maps) for more.

The method will return a maplist file, which format is [described here](https://agilepy.readthedocs.io/en/latest/manual/products.html#maplist-file). 

In [11]:
mapfilepath = ag.generateMaps()

2020-02-26 17:53:00,061 [INFO    ] [CtsMapGenerator] Science tool called!
2020-02-26 17:53:01,639 [INFO    ] [AGAnalysis] Science tool ctsMapGenerator produced ['/home/rt/.conda/envs/agilepy/agiletools/agilepy-test-data/unittesting-output/jupyter/maps/quickstart_EMIN00100_EMAX10000_01.cts.gz']
2020-02-26 17:53:01,641 [INFO    ] [ExpMapGenerator] Science tool called!
2020-02-26 17:53:11,399 [INFO    ] [AGAnalysis] Science tool expMapGenerator produced ['/home/rt/.conda/envs/agilepy/agiletools/agilepy-test-data/unittesting-output/jupyter/maps/quickstart_EMIN00100_EMAX10000_01.exp.gz']
2020-02-26 17:53:11,402 [INFO    ] [GasMapGenerator] Science tool called!
2020-02-26 17:53:12,781 [INFO    ] [AGAnalysis] Science tool gasMapGenerator produced ['/home/rt/.conda/envs/agilepy/agiletools/agilepy-test-data/unittesting-output/jupyter/maps/quickstart_EMIN00100_EMAX10000_01.gas.gz']
2020-02-26 17:53:12,783 [INFO    ] [IntMapGenerator] Science tool called!
2020-02-26 17:53:13,419 [INFO    ] [AGAna

Let's print the content of the mapfile:

In [12]:
with open(mapfilepath) as f:
    print(f.read())

/home/rt/.conda/envs/agilepy/agiletools/agilepy-test-data/unittesting-output/jupyter/maps/quickstart_EMIN00100_EMAX10000_01.cts.gz /home/rt/.conda/envs/agilepy/agiletools/agilepy-test-data/unittesting-output/jupyter/maps/quickstart_EMIN00100_EMAX10000_01.exp.gz /home/rt/.conda/envs/agilepy/agiletools/agilepy-test-data/unittesting-output/jupyter/maps/quickstart_EMIN00100_EMAX10000_01.gas.gz 30 -1 -1



### Parsing the mapfile

In [13]:
maps = ag.getSkyMaps()

In [14]:
print("Number of rows:",len(maps))

Number of rows: 1


In [15]:
ctsMap = maps[0][0]
ctsMap

'/home/rt/.conda/envs/agilepy/agiletools/agilepy-test-data/unittesting-output/jupyter/maps/quickstart_EMIN00100_EMAX10000_01.cts.gz'

In [16]:
expMap = maps[0][1]
expMap

'/home/rt/.conda/envs/agilepy/agiletools/agilepy-test-data/unittesting-output/jupyter/maps/quickstart_EMIN00100_EMAX10000_01.exp.gz'

In [17]:
gasMap = maps[0][2]
gasMap

'/home/rt/.conda/envs/agilepy/agiletools/agilepy-test-data/unittesting-output/jupyter/maps/quickstart_EMIN00100_EMAX10000_01.gas.gz'

## Starting an maximum likelyhood estimation analysis

Again, the behaviour of this analysis can be configured by sevaral configuration options. Check the [*'mle'* section of the documentation](https://agilepy.readthedocs.io/en/latest/manual/configuration_file.html#mle) for more.

The [mle()](https://agilepy.readthedocs.io/en/latest/api/analysis_api.html#api.AGAnalysis.AGAnalysis.mle) performs a maximum likelyhood estimation analysis and it produces one or more files with the '*.source*' extension. The '*.source*' files have a fixed format described [here](https://agilepy.readthedocs.io/en/latest/manual/products.html#source-file).

The mle analysis output is also saved inside the corresponding sources in the *SourcesLibrary* object. 

In [18]:
source_files = ag.mle()

2020-02-26 17:54:17,366 [INFO    ] [SourcesLibrary] File /home/rt/.conda/envs/agilepy/agiletools/agilepy-test-data/unittesting-output/jupyter/sourceLibrary00000.txt has been produced
2020-02-26 17:54:17,368 [INFO    ] [Multi] Science tool called!
2020-02-26 17:54:20,387 [INFO    ] [AGAnalysis] AG_multi produced: ['/home/rt/.conda/envs/agilepy/agiletools/agilepy-test-data/unittesting-output/jupyter/mle/quickstart0000_2AGLJ2021+4029.source', '/home/rt/.conda/envs/agilepy/agiletools/agilepy-test-data/unittesting-output/jupyter/mle/quickstart0000_2AGLJ2021+3654.source']
2020-02-26 17:54:20,392 [INFO    ] [AGAnalysis] Took 3.027477 seconds.


In [19]:
for sf in source_files:
    print(sf)

/home/rt/.conda/envs/agilepy/agiletools/agilepy-test-data/unittesting-output/jupyter/mle/quickstart0000_2AGLJ2021+4029.source
/home/rt/.conda/envs/agilepy/agiletools/agilepy-test-data/unittesting-output/jupyter/mle/quickstart0000_2AGLJ2021+3654.source


In [22]:
sources = ag.selectSources("sqrtTS > 0")


-----------------------------------------------------------
Source name: 2AGLJ2021+4029 (PointSource) => sqrt(ts): 1.72593
  * Position:
	- start_pos: (78.2375, 2.12298)
  * Spectrum: (PLExpCutoff)
	- flux: 1.193e-06
	- index: 1.75
	- cutoffEnergy: 3307.63
  * Free params: flux
  * Multi analysis:
	- flux: 9.68426e-07 +- 6.75116e-07
	- upper limit: 2.56733e-06
	- ergLog: 1.56513e-10 +- 1.0911e-10
	- galCoeff: [0.471781]
	- isoCoeff: [10.2321]
-----------------------------------------------------------

-----------------------------------------------------------
Source name: 2AGLJ2021+3654 (PointSource) => sqrt(ts): 1.17321
  * Position:
	- start_pos: (75.2562, 0.151831)
  * Spectrum: (PLExpCutoff)
	- flux: 7.089e-07
	- index: 1.38
	- cutoffEnergy: 960.59
  * Free params: flux
  * Multi analysis:
	- flux: 5.23926e-07 +- 5.19933e-07
	- upper limit: 1.79627e-06
	- ergLog: 8.46749e-11 +- 8.40296e-11
	- galCoeff: [0.549708]
	- isoCoeff: [8.66681]
-------------------------------------------

## Generating a light curve
The [lightCurve()](https://agilepy.readthedocs.io/en/latest/api/analysis_api.html#api.AGAnalysis.AGAnalysis.lightCurve) method will generate a csv file containing the data for a light curve plot.

In [23]:
lightCurveDataFile = ag.lightCurve("2AGLJ2021+3654", binsize = 86400, processes=1)

2020-02-26 17:56:21,148 [INFO    ] [AGAnalysis] Number of processes: 1, Number of bins per process 3
2020-02-26 17:56:21,150 [INFO    ] [CtsMapGenerator] Science tool called!
2020-02-26 17:56:22,190 [INFO    ] [AGAnalysis] Science tool ctsMapGenerator produced ['/home/rt/.conda/envs/agilepy/agiletools/agilepy-test-data/unittesting-output/jupyter/lc/bin_456361778_456448178/maps/lc_analysis_EMIN00100_EMAX10000_01.cts.gz']
2020-02-26 17:56:22,192 [INFO    ] [ExpMapGenerator] Science tool called!
2020-02-26 17:56:27,486 [INFO    ] [AGAnalysis] Science tool expMapGenerator produced ['/home/rt/.conda/envs/agilepy/agiletools/agilepy-test-data/unittesting-output/jupyter/lc/bin_456361778_456448178/maps/lc_analysis_EMIN00100_EMAX10000_01.exp.gz']
2020-02-26 17:56:27,488 [INFO    ] [GasMapGenerator] Science tool called!
2020-02-26 17:56:28,906 [INFO    ] [AGAnalysis] Science tool gasMapGenerator produced ['/home/rt/.conda/envs/agilepy/agiletools/agilepy-test-data/unittesting-output/jupyter/lc/bin

Let's inspect the output file:

In [24]:
with open(lightCurveDataFile) as lcd:
    file_contents = lcd.read()
print(file_contents)

t_start t_end sqrt_ts flux flux_err flux_ul
456361778.0 456448178.0 0.526542 0.526542 7.65885e-07 2.35674e-06
456448178.0 456534578.0 1.13063 1.13063 7.46566e-07 2.64912e-06
456534578.0 456620978.0 0.0 0.0 2.31656e-06 9.2693e-06



## Interacting with the SourcesLibrary

You can interact with the SourcesLibrary throught the *AGAnalysis* class that provides you three useful methods.

### Get all the sources
You can obtain a list of all the sources with [getSources()](https://agilepy.readthedocs.io/en/latest/api/analysis_api.html#api.AGAnalysis.AGAnalysis.getSources).

In [22]:
sources = ag.getSources()

In [23]:
len(sources)

2

In [24]:
for source in sources:
    print(source)


-----------------------------------------------------------
Source name: 2AGLJ2021+4029 (PointSource) => sqrt(ts): 1.72593
  * Position:
	- start_pos: (78.2375, 2.12298)
  * Spectrum: (PLExpCutoff)
	- flux: 1.193e-06
	- index: 1.75
	- cutoffEnergy: 3307.63
  * Free params: flux
  * Multi analysis:
	- flux: 9.68426e-07 +- 6.75116e-07
	- upper limit: 2.56733e-06
	- ergLog: 1.56513e-10 +- 1.0911e-10
	- galCoeff: [0.471781]
	- isoCoeff: [10.2321]
-----------------------------------------------------------

-----------------------------------------------------------
Source name: 2AGLJ2021+3654 (PointSource) => sqrt(ts): 1.17321
  * Position:
	- start_pos: (75.2562, 0.151831)
  * Spectrum: (PLExpCutoff)
	- flux: 7.089e-07
	- index: 1.38
	- cutoffEnergy: 960.59
  * Free params: flux
  * Multi analysis:
	- flux: 5.23926e-07 +- 5.19933e-07
	- upper limit: 1.79627e-06
	- ergLog: 8.46749e-11 +- 8.40296e-11
	- galCoeff: [0.549708]
	- isoCoeff: [8.66681]
-------------------------------------------

### Selecting sources

You can perform a query with [selectSources(selection)](https://agilepy.readthedocs.io/en/latest/api/analysis_api.html#api.AGAnalysis.AGAnalysis.selectSources).

The sources can be selected with the '*selection*' argument, supporting either lambda functions and boolean expression strings.

The selection criteria can be expressed using the following Source class’s parameters:

* name: the unique code identifying the source.
* dist: the distance of the source from the center of the maps.
* flux: the flux value.
* sqrtTS: the radix square of the ts.

The *sqrtTS* selection parameter can be used only after a call to mle().

#### Lamba selection

In [25]:
sources = ag.selectSources(lambda name, dist, flux : name == "2AGLJ2021+4029" and dist > 2.5 and flux > 0)


-----------------------------------------------------------
Source name: 2AGLJ2021+4029 (PointSource) => sqrt(ts): 1.72593
  * Position:
	- start_pos: (78.2375, 2.12298)
  * Spectrum: (PLExpCutoff)
	- flux: 1.193e-06
	- index: 1.75
	- cutoffEnergy: 3307.63
  * Free params: flux
  * Multi analysis:
	- flux: 9.68426e-07 +- 6.75116e-07
	- upper limit: 2.56733e-06
	- ergLog: 1.56513e-10 +- 1.0911e-10
	- galCoeff: [0.471781]
	- isoCoeff: [10.2321]
-----------------------------------------------------------


In [26]:
for source in sources:
    print(source)


-----------------------------------------------------------
Source name: 2AGLJ2021+4029 (PointSource) => sqrt(ts): 1.72593
  * Position:
	- start_pos: (78.2375, 2.12298)
  * Spectrum: (PLExpCutoff)
	- flux: 1.193e-06
	- index: 1.75
	- cutoffEnergy: 3307.63
  * Free params: flux
  * Multi analysis:
	- flux: 9.68426e-07 +- 6.75116e-07
	- upper limit: 2.56733e-06
	- ergLog: 1.56513e-10 +- 1.0911e-10
	- galCoeff: [0.471781]
	- isoCoeff: [10.2321]
-----------------------------------------------------------


#### Boolean string selection

In [27]:
sources = ag.selectSources('name == "2AGLJ2021+4029" AND dist > 0 AND flux > 0')


-----------------------------------------------------------
Source name: 2AGLJ2021+4029 (PointSource) => sqrt(ts): 1.72593
  * Position:
	- start_pos: (78.2375, 2.12298)
  * Spectrum: (PLExpCutoff)
	- flux: 1.193e-06
	- index: 1.75
	- cutoffEnergy: 3307.63
  * Free params: flux
  * Multi analysis:
	- flux: 9.68426e-07 +- 6.75116e-07
	- upper limit: 2.56733e-06
	- ergLog: 1.56513e-10 +- 1.0911e-10
	- galCoeff: [0.471781]
	- isoCoeff: [10.2321]
-----------------------------------------------------------


In [28]:
for source in sources:
    print(source)


-----------------------------------------------------------
Source name: 2AGLJ2021+4029 (PointSource) => sqrt(ts): 1.72593
  * Position:
	- start_pos: (78.2375, 2.12298)
  * Spectrum: (PLExpCutoff)
	- flux: 1.193e-06
	- index: 1.75
	- cutoffEnergy: 3307.63
  * Free params: flux
  * Multi analysis:
	- flux: 9.68426e-07 +- 6.75116e-07
	- upper limit: 2.56733e-06
	- ergLog: 1.56513e-10 +- 1.0911e-10
	- galCoeff: [0.471781]
	- isoCoeff: [10.2321]
-----------------------------------------------------------


### Free or Fix sources parameters

You can let some of the source parameters free to vary: in order to decide which parameters are free to vary and which parameters are fixed you can use the [freeSources(selection, parameterName, free)](https://agilepy.readthedocs.io/en/latest/api/analysis_api.html#api.AGAnalysis.AGAnalysis.freeSources) method.

The sources can be selected with the '*selection*' argument.

The '*parameterName*' argument is the name of the parameter you want to fix or let free to vary.

The '*free*' argument is a boolean:
* True: the parameter will be free to vary
* False: the parameter will be fixed.

The method returns the list of sources affected by the update.

In [29]:
sources = ag.freeSources('name == "2AGLJ2021+4029" AND dist > 0 AND flux > 0', "flux", False)


-----------------------------------------------------------
Source name: 2AGLJ2021+4029 (PointSource) => sqrt(ts): 1.72593
  * Position:
	- start_pos: (78.2375, 2.12298)
  * Spectrum: (PLExpCutoff)
	- flux: 1.193e-06
	- index: 1.75
	- cutoffEnergy: 3307.63
  * Free params: flux
  * Multi analysis:
	- flux: 9.68426e-07 +- 6.75116e-07
	- upper limit: 2.56733e-06
	- ergLog: 1.56513e-10 +- 1.0911e-10
	- galCoeff: [0.471781]
	- isoCoeff: [10.2321]
-----------------------------------------------------------


In [30]:
print(sources[0])


-----------------------------------------------------------
Source name: 2AGLJ2021+4029 (PointSource) => sqrt(ts): 1.72593
  * Position:
	- start_pos: (78.2375, 2.12298)
  * Spectrum: (PLExpCutoff)
	- flux: 1.193e-06
	- index: 1.75
	- cutoffEnergy: 3307.63
  * Free params: 
  * Multi analysis:
	- flux: 9.68426e-07 +- 6.75116e-07
	- upper limit: 2.56733e-06
	- ergLog: 1.56513e-10 +- 1.0911e-10
	- galCoeff: [0.471781]
	- isoCoeff: [10.2321]
-----------------------------------------------------------


In [31]:
sources = ag.freeSources('name == "2AGLJ2021+4029" AND dist > 0 AND flux > 0', "pos", True)


-----------------------------------------------------------
Source name: 2AGLJ2021+4029 (PointSource) => sqrt(ts): 1.72593
  * Position:
	- start_pos: (78.2375, 2.12298)
  * Spectrum: (PLExpCutoff)
	- flux: 1.193e-06
	- index: 1.75
	- cutoffEnergy: 3307.63
  * Free params: 
  * Multi analysis:
	- flux: 9.68426e-07 +- 6.75116e-07
	- upper limit: 2.56733e-06
	- ergLog: 1.56513e-10 +- 1.0911e-10
	- galCoeff: [0.471781]
	- isoCoeff: [10.2321]
-----------------------------------------------------------


In [32]:
print(sources[0])


-----------------------------------------------------------
Source name: 2AGLJ2021+4029 (PointSource) => sqrt(ts): 1.72593
  * Position:
	- start_pos: (78.2375, 2.12298)
  * Spectrum: (PLExpCutoff)
	- flux: 1.193e-06
	- index: 1.75
	- cutoffEnergy: 3307.63
  * Free params: pos
  * Multi analysis:
	- flux: 9.68426e-07 +- 6.75116e-07
	- upper limit: 2.56733e-06
	- ergLog: 1.56513e-10 +- 1.0911e-10
	- galCoeff: [0.471781]
	- isoCoeff: [10.2321]
	- L_peak: 78.2375
	- B_peak: 2.12298
	- distFromStartPos: 8.53774e-07
	- L: -1.0
	- B: -1.0
	- distFromStartPos: -1.0
	- r: -1.0
	- a: -1.0
	- b: -1.0
	- phi: -1.0
-----------------------------------------------------------


### Deleting sources

You can delete sources with [deleteSources(selection)](https://agilepy.readthedocs.io/en/latest/api/analysis_api.html#api.AGAnalysis.AGAnalysis.deleteSources).

The method returns the list of deleted sources.

In [ ]:
deletedSources = ag.deleteSources('flux > 0')

In [ ]:
for s in deletedSources:
    print(s)

We can check that the sources are deleted.

In [ ]:
sources = ag.getSources()

In [ ]:
len(sources)

### Adding a source
You can add a new source with [addSource(sourceName, sourceDict)](https://agilepy.readthedocs.io/en/latest/api/analysis_api.html#api.AGAnalysis.AGAnalysis.addSource).

You have to pass a dictionary containing the source's data.

In [ ]:
newSourceDict = {
    "glon" : 6.16978,
    "glat": -0.0676943,
    "spectrumType" : "LogParabola",
    "flux": 35.79e-08,
    "curvature": 0.682363
}
if ag.addSource("2AGLJ1801-2334", newSourceDict):
    print("Source loaded")

In [ ]:
if not ag.addSource("2AGLJ1801-2334", newSourceDict):
    print("Source NOT loaded, it is already present in the Sources Library")

## Updating the configuration options

You can update the configuration options using the [setOptions(**kwargs)](https://agilepy.readthedocs.io/en/latest/api/analysis_api.html#api.AGAnalysis.AGAnalysis.setOptions) method.

In [ ]:
ag.setOptions(binsize=0.6, mapsize=60, energybins=[[100, 1000], [1000, 3000]])

You can print on the console the configuration options values with the printOptions(section) method.

The section argument is optional but you can use to print a subset of the options.

In [ ]:
ag.printOptions("maps")

## Displaying the sky maps

You can display the sky maps using the [displaySkyMap()]() method.

In [ ]:
%matplotlib notebook

In [ ]:
ag.displaySkyMap(ctsMap, smooth=False)

You can also smooth the image

In [ ]:
ag.displaySkyMap(ctsMap, smooth=True, sigma=5)

Finally you can save it on disk

In [ ]:
outputfile = ag.displaySkyMap(expMap, smooth=True, sigma=5, saveImage=True)
print(outputfile,"saved.")